$\textbf{01: Initial Cohort Design}$

- Includes injury data from __two sources__:
    - [Tommy John Surgery List](https://docs.google.com/spreadsheets/d/1gQujXQQGOVNaiuwSN680Hq-FDVsCwvN-3AazykOBON0/edit?gid=0#gid=0)
    - [FanGraphs Injury Database](https://www.fangraphs.com/roster-resource/injury-report?groupby=all&timeframe=all&season=2020&injury=)


In [1]:
import pandas as pd

$\textbf{Tommy John Surgery List}$

- Filters: 
    - Injuries from ball tracking era (2015 - 2025, since spin was added in 2015)
    - Injuries during the season (February - November, inclusive) _with preceding ball tracking data_

In [2]:
# load initial cohort (drop player names) --> 2,006 injuries
tj_cohort = pd.read_csv('cohort_data/tj_aug_5_2025.csv').drop(
    columns=[
        'Player', 
        'Team', 
        'Country', 
        'High School', 
        'College(s)',
        'Return Date (same level)',
        'Recovery Time (months)',
        'Surgeon(s)',
        'Post-TJ MLB G',
        'Post-TJ MLB IP/PA'
        ]
    )

# rename columns
tj_cohort.rename(columns={'TJ Surgery Date': 'injury_date'}, inplace=True)
tj_cohort['injury_date'] = pd.to_datetime(tj_cohort['injury_date'], format='%m/%d/%y').dt.date

# clean mlbamid
tj_cohort.dropna(subset=['mlbamid'], inplace=True)                         # 35 missing, mostly new draftees or signees
tj_cohort['mlbamid'] = tj_cohort['mlbamid'].astype(int).astype('str')

In [3]:
# filters --> 1,052 injuries before ball tracking merge
tj_cohort_filtered = tj_cohort[(tj_cohort['Year'] >= 2015) & (tj_cohort['Year'] <= 2025)]                               # years: 2015-2024 (start of ball tracking w/ spin)
tj_cohort_filtered = tj_cohort_filtered[(tj_cohort_filtered['Month'] >= 2) & (tj_cohort_filtered['Month'] <= 11)]       # months: May-Oct (in-season)

$\textbf{FanGraphs Injury List}$

- Spans from 2020 - 2025
- __Note__: Last update for 2025 was 5/19/25
- Filters:
    - Injuries to pitchers (`RP` or `SP`)
    - Elbow injuries (`injury_type` contains `elbow` or `Tommy John`)
    - Injuries during the season (May - October, inclusive); this is done since ACWR requires 30 days

In [4]:
import os
pd.options.mode.chained_assignment = None

In [6]:
# load fangraphs data files
fg_files = [f for f in os.listdir('cohort_data') if f.startswith('fg_')]

# iterate through files
fg_data = []
for f in fg_files:
    fg_data.append(pd.read_csv(f'cohort_data/{f}'))

# concatenate data --> 4,531 injuries
fg_data = pd.concat(fg_data).reset_index(drop=True)
fg_data.drop(columns=['Status'], inplace=True)
fg_data.drop_duplicates(subset=['player_name', 'injury_type', 'injury_date'], inplace=True)

In [7]:
# filter to only pitchers
fg_data_filtered = fg_data[fg_data['position'].isin(['SP', 'RP'])]                                          # only pitchers

# elbow-related injuries (including TJ to cross-reference w/ TJ list)
    # leaves 323 injuries
fg_data_filtered['injury_type'] = fg_data_filtered['injury_type'].str.lower().str.replace(' ', '_')
fg_data_filtered = fg_data_filtered[
    (fg_data_filtered['injury_type'].str.contains('elbow', case=False) | fg_data_filtered['injury_type'].str.contains('tommy', case=False)) & 
    (~fg_data_filtered['injury_type'].str.contains('fractured', case=False))
]  

# date filtering
fg_data_filtered.loc[:, 'injury_date'] = pd.to_datetime(
    fg_data_filtered['injury_date'], 
    format='%m/%d/%y', 
    errors='coerce'
).dt.date
fg_data_filtered['month'] = fg_data_filtered.loc[:, 'injury_date'].apply(lambda x: x.month)
fg_data_filtered = fg_data_filtered[(fg_data_filtered['month'] >= 2) & (fg_data_filtered['month'] <= 11)]

$\textit{FanGraphs Player ID Lookup}$

- Uses the Statcast API function to lookup ID given name
- Then filters based on team
- Will take a few moments upon first call, but then runs quickly (caches table)
- __Note__: If players don't return an ID, it's manually looked up

In [8]:
from pybaseball import playerid_lookup      # need the `key_mlbam` value

In [9]:
# iterate through FG names
manual_lookup = {}
for name in fg_data_filtered['player_name'].unique():
    first, last = name.split(' ')[0], name.split(' ')[1]                                                    # split first and last name
    try:
        bam_id = playerid_lookup(last, first)['key_mlbam'].values[0].astype(int).astype(str)                # get mlbamid
        fg_data_filtered.loc[fg_data_filtered['player_name'] == name, 'mlbamid'] = bam_id                   # assign mlbamid to df

    except Exception as e:
        print(f'Adding {name} to manual lookup dictionary.')
        manual_lookup[name] = None

Gathering player lookup table. This may take a moment.
Adding Matt Bowman to manual lookup dictionary.
Adding Ricardo Sánchez to manual lookup dictionary.
Adding Nestor Cortes to manual lookup dictionary.
Adding Kwang Hyun Kim to manual lookup dictionary.
Adding Jimmie Sherfy to manual lookup dictionary.
Adding Matthew Boyd to manual lookup dictionary.
Adding J.B. Bukauskas to manual lookup dictionary.
Adding Adrian Morejon to manual lookup dictionary.
Adding Sam Delaplane to manual lookup dictionary.
Adding Michel Baez to manual lookup dictionary.
Adding Dylan File to manual lookup dictionary.
Adding Daniel Tillo to manual lookup dictionary.
Adding Sandy Alcantara to manual lookup dictionary.
Adding Angel Perdomo to manual lookup dictionary.
Adding José De León to manual lookup dictionary.
Adding Freddy Pacheco to manual lookup dictionary.
Adding A.J. Puk to manual lookup dictionary.
Adding Adrián Martínez to manual lookup dictionary.
Adding Bryce Montes de Oca to manual lookup dictio

In [10]:
# update manual lookups
manual_lookup['Matt Bowman'] = '621199'
manual_lookup['Ricardo Sánchez'] = '645307'
manual_lookup['Nestor Cortes'] = '641482'
manual_lookup['Kwang Hyun Kim'] = '547942'
manual_lookup['Jimmie Sherfy'] = '642073'
manual_lookup['Matthew Boyd'] = '571510'
manual_lookup['J.B. Bukauskas'] = '656266'
manual_lookup['Adrian Morejon'] = '670970'
manual_lookup['Sam Delaplane'] = '676845'
manual_lookup['Michel Baez'] = '673258'
manual_lookup['Dylan File'] = '668766'
manual_lookup['Daniel Tillo'] = '664700'
manual_lookup['Sandy Alcantara'] = '645261'
manual_lookup['Angel Perdomo'] = '622780'
manual_lookup['José De León'] = '592254'
manual_lookup['Freddy Pacheco'] = '681643'
manual_lookup['A.J. Puk'] = '640462'
manual_lookup['Adrián Martínez'] = '661309'
manual_lookup['Bryce Montes de Oca'] = '656755'
manual_lookup['Jose Alvarez'] = '501625'
manual_lookup['Jorge Alcala'] = '660896'
manual_lookup['Michael King'] = '650633'
manual_lookup['Vladimir Gutierrez'] = '661269'
manual_lookup['Hyun Jin Ryu'] = '547943'
manual_lookup['José Fermin'] = '820862'
manual_lookup['J.T. Ginn'] = '669372'
manual_lookup['Luis F. Ortiz'] = '656814'
manual_lookup['Josh Simpson'] = '681006'

# iterate through and set in dataframe
for name, mlbamid in manual_lookup.items():
    fg_data_filtered.loc[fg_data_filtered['player_name'] == name, 'mlbamid'] = mlbamid


$\textit{Cross Reference Data Sources}$ 

- Check if any FanGraphs TJ's are not in other data source (__9 total__)
- Determine how many additional injuries are added by FG (__303 total__)

In [11]:
# fangraphs
fg_tommy_johns = list(fg_data_filtered[fg_data_filtered['injury_type'].str.contains('tommy', case=False)]['mlbamid'])       # TJ surgeries
fg_other_elbow = list(fg_data_filtered[fg_data_filtered['injury_type'].str.contains('elbow', case=False)]['mlbamid'])       # other elbow injuries

# tj_cohort
tj_list = list(tj_cohort_filtered['mlbamid'].astype(str))                                                                   # TJ surgeries

# check for TJ surgeries from FG that are missing from TJ list
missing_tj = [x for x in fg_tommy_johns if x not in tj_list]

$\textit{Get Unique Injuries and Dates}$

- __1,380__ total injuries w/ date and injury type
- __Note__: This will have to be filtered to those who have ball tracking data (necessarily smaller given minor league players in dataset)

In [16]:
# TJ list
tj_cohort_filtered['injury_type'] = 'tommy_john_surgery'
tj_list_injuries = tj_cohort_filtered[['mlbamid', 'injury_date', 'injury_type']]

# FG list
fg_list_injuries = fg_data_filtered[['mlbamid', 'injury_date', 'injury_type']]

# combine lists, drop duplicates --> 1,282 total injuries
combined_list = pd.concat([tj_list_injuries, fg_list_injuries])
combined_list.drop_duplicates(subset=['mlbamid', 'injury_date'], inplace=True)
combined_list.sort_values(by='injury_date', inplace=True)

In [19]:
# save final cohort
combined_list.to_csv('cohort_data/d3_final_cohort.csv', index=False)

$\textbf{Ball Tracking Data}$

In [20]:
# columns present in 2008-2014 data:
    # release_speed
    # release_pos_x, release_pos_z
    # pfx_x, pfx_z
    # vx0, vy0, vz0
    # ax, ay, az
    # api_break_x_arm, api_break_z_with_gravity

In [21]:
from pybaseball import statcast_pitcher

In [22]:
# date format: YYYY-MM-DD
test = statcast_pitcher(player_id='623437', start_dt='2024-01-01', end_dt='2024-05-01')

Gathering Player Data


$\textbf{Sandbox: Examples}$

In [ ]:
# 2008 MLB pitcher
test = statcast_pitcher(start_dt='2008-04-01', end_dt='2008-11-01', player_id=150414)

Gathering Player Data
